# Weather Data Set


In this notebook we import the weather data, preprocess if is needed and save as a new Data Frame. 


### Historical Data 

For training and testing the model we use the historicla weather data related to our energy consumption data provided. 

For that, we'll use the API provided by [AT-Wetter](http://at-wetter.tk/index.php?men=api). We'll use this one, and not the OpenWeather API since if we want to get the historical data depth more than a week (we need a year) we'll have to use several queries. 


#### API 

This API don't need a Key or other autentication. It's located on the AT-Server in `http://at-wetter.tk/api/v1/.`
The instruction to use it are in their [website](http://at-wetter.tk/index.php?men=api) 

#### Example 

To query **all the available stations** we need a call following the format: 
`/api/v1/[field]/[YYYY-MM-DD]/[day count]` 

A query call to `http://at-wetter.tk/api/v1/t/2014-08-01/2`  returns Temperature Data for every available station for 2014-08-01, 2 days in the past. 



To query just **one specific stations** we use: 

`/api/v1/station/[station]/[field]|[all]/[YYYY-MM-DD]/[day count]`

This will return the `[field]` from the specified date `[YYYY-MM-DD]` and prints data `[day count]` days in the past for the specified stations `[station]`



### Calling the API


In [1]:
import requests as rq
import json
import datetime as dt
import pandas as pd
import numpy as np

* See the available stations 

In [9]:
URL ="http://at-wetter.tk/api/v1/stations"
response = rq.request("GET", URL)
print(response.text)

'11157';'Aigen im Ennstal';'640';'m'
'11244';'Bad Gleichenberg';'280';'m'
'11101';'Bregenz';'424';'m'
'11190';'Eisenstadt';'184';'m'
'11';'Feuerkogel';'1618';'m'
'11155';'Feuerkogel';'1618';'m'
'11240';'Graz/Flughafen';'340';'m'
'11121';'Innsbruck';'579';'m'
'11331';'Klagenfurt/Flughafen';'447';'m'
'11012';'Kremsmünster';'383';'m'
'11012';'Kremsm�nster';'383';'m'
'11130';'Kufstein';'495';'m'
'11204';'Lienz';'659';'m'
'11010';'Linz/Hörsching';'298';'m'
'11010';'Linz/H�rsching';'298';'m'
'11171';'Mariazell';'866';'m'
'11126';'Patscherkofel';'2247';'m'
'11022';'Retz';'320';'m'
'11150';'Salzburg';'430';'m'
'11343';'Sonnblick';'3105';'m'
'11389';'St. Pölten';'270';'m'
'11389';'St. P�lten';'270';'m'
'11265';'Villacher Alpe';'2140';'m'
'11035';'Wien/Hohe Warte';'203';'m'
'11036';'Wien/Schwechat';'183';'m'



* Calculate de day count from Building Energy Data set 

We need the day count in order to ask these exactly days to weather API to match it with our energy data set.
We could do it manually, but in order to automated all the steps we'll calculated with a python script 

In [13]:
import pandas as pd 
DataFrame = pd.read_excel("./DataSets/Raw_Building_energy_DataSet.xlsx", header=1)
DataFrame.head()

#Calculate the Delta Time 

start_date = dt.datetime.strptime(DataFrame['Time'][0], "%Y-%m-%d %H:%M:%S")
end_date = dt.datetime.strptime(DataFrame['Time'][len(DataFrame)-1], "%Y-%m-%d %H:%M:%S")
day_count = end_date - start_date
print(day_count)

280 days, 5:00:00


As the `[daycount]` field for API Query doesn't allow hours, if we have any hours of difference between the start day and the end day, we add an extra day. Then in the data frame matching we'll delete the extra hours for the extra day. 

In [14]:
dayCount=0 
if day_count.seconds //3600 > 0: 
    dayCount = day_count.days + 1
else: 
    dayCount = day_count.days
print(dayCount)

281


* Download the weather data

In [22]:
station = '11240' #GRAZ 
field = 'all'  #Temperatuer
days_count = str(day_count.days)

URL = "http://at-wetter.tk/api/v1/station/"
URL += station 
URL += "/" + field
URL += "/" + str(end_date.year) + "-" + str(end_date.month) + "-" + str(end_date.day)
URL += "/" + str(dayCount)
response = rq.request("GET", URL)

print(URL)
#print(response.text)
type(response.text)

http://at-wetter.tk/api/v1/station/11240/all/2021-11-3/281


str

In [ ]:
#response.json(encoding="utf-8")
#print(response.text)

* Saving the data into a Data Frame 

Since the data can't be downloaded in JSON format, we first split the whole text into lineas, and then elements. 

In [23]:
lines = response.text.splitlines()
ColumnNames = [ (col.replace("'",'')) for col in lines[0].split(";") ]
WeatDataFrame = pd.DataFrame(columns=ColumnNames, index=range(len(lines)-2)) # -1 cause the columnames and -1 again cuase it starts from 0 

The index of the daframe starts from 0 but the first line of response.text is the ColumnName row. So we start from 0 for the DataFrame index and for idx+1 for lineas

In [24]:
for idx in range(len(lines)-2):  
    WeatDataFrame.iloc[idx] = [ (col.replace("'",'')) for col in lines[idx+1].split(";") ]
WeatDataFrame

,station,name,hoehe,datum,zeit,t,tp,rf,wr,wg,wsr,wsg,regen,ldred,ldstat,sonne,timestamp
0,11240,Graz/Flughafen,340,2021-01-26,00:00,-3.1,-4.0,93,220,1.8,null,7.6,0.0,1010.3,965.2,0,2021-01-26 00:10:10
1,11240,Graz/Flughafen,340,2021-01-26,01:00,-3.3,-3.9,96,360,3.6,null,5.4,0.0,1010.9,965.8,0,2021-01-26 01:10:09
2,11240,Graz/Flughafen,340,2021-01-26,02:00,-2.8,-3.5,95,260,5.4,null,7.6,0.0,1011.1,966.0,0,2021-01-26 02:10:08
3,11240,Graz/Flughafen,340,2021-01-26,03:00,-3.8,-4.2,97,180,5.4,null,7.6,0.0,1011.4,966.2,0,2021-01-26 03:10:08
4,11240,Graz/Flughafen,340,2021-01-26,04:00,-4.7,-5.3,96,280,1.8,null,9.4,0.0,1012.0,966.6,0,2021-01-26 04:10:09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6693,11240,Graz/Flughafen,340,2021-11-03,18:00,7.9,7.3,96,160,5.4,null,9.4,0.2,1005.3,962.2,0,2021-11-03 18:10:10
6694,11240,Graz/Flughafen,340,2021-11-03,19:00,7.6,7.0,96,230,5.4,null,9.4,0.5,1004.7,961.6,0,2021-11-03 19:10:08
6695,11240,Graz/Flughafen,340,2021-11-03,20:00,7.6,7.0,96,0,1.8,null,9.4,0.9,1004.2,961.1,0,2021-11-03 20:10:12
6696,11240,Graz/Flughafen,340,2021-11-03,21:00,7.9,7.5,97,80,3.6,null,7.6,1.0,1003.8,960.8,0,2021-11-03 21:10:08


* Changing the datum and zeit column into a newone with datetime type object

In [25]:
WeatDataFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6698 entries, 0 to 6697
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   station    6698 non-null   object
 1   name       6698 non-null   object
 2   hoehe      6698 non-null   object
 3   datum      6698 non-null   object
 4   zeit       6698 non-null   object
 5   t          6698 non-null   object
 6   tp         6698 non-null   object
 7   rf         6698 non-null   object
 8   wr         6698 non-null   object
 9   wg         6698 non-null   object
 10  wsr        6698 non-null   object
 11  wsg        6698 non-null   object
 12  regen      6698 non-null   object
 13  ldred      6698 non-null   object
 14  ldstat     6698 non-null   object
 15  sonne      6698 non-null   object
 16  timestamp  6698 non-null   object
dtypes: object(17)
memory usage: 889.7+ KB


In [26]:
WeatDataFrame["datum"][:]

0       2021-01-26
1       2021-01-26
2       2021-01-26
3       2021-01-26
4       2021-01-26
           ...    
6693    2021-11-03
6694    2021-11-03
6695    2021-11-03
6696    2021-11-03
6697    2021-11-03
Name: datum, Length: 6698, dtype: object

* Converting string type date to datetime type objects 

In [27]:
from datetime import datetime as dt 

dates=[] 
for idx in range(len(WeatDataFrame["datum"])): 
    str_date = str(WeatDataFrame["datum"][idx]) + " " + str(WeatDataFrame["zeit"][idx])
    dt = dt.strptime(str_date, "%Y-%m-%d %H:%M")
    dates.append(dt)

print(len(dates), len(WeatDataFrame.index))
WeatDataFrame.insert(0,"Time",dates)

6698 6698


In [28]:
WeatDataFrame

,Time,station,name,hoehe,datum,zeit,t,tp,rf,wr,wg,wsr,wsg,regen,ldred,ldstat,sonne,timestamp
0,2021-01-26 00:00:00,11240,Graz/Flughafen,340,2021-01-26,00:00,-3.1,-4.0,93,220,1.8,null,7.6,0.0,1010.3,965.2,0,2021-01-26 00:10:10
1,2021-01-26 01:00:00,11240,Graz/Flughafen,340,2021-01-26,01:00,-3.3,-3.9,96,360,3.6,null,5.4,0.0,1010.9,965.8,0,2021-01-26 01:10:09
2,2021-01-26 02:00:00,11240,Graz/Flughafen,340,2021-01-26,02:00,-2.8,-3.5,95,260,5.4,null,7.6,0.0,1011.1,966.0,0,2021-01-26 02:10:08
3,2021-01-26 03:00:00,11240,Graz/Flughafen,340,2021-01-26,03:00,-3.8,-4.2,97,180,5.4,null,7.6,0.0,1011.4,966.2,0,2021-01-26 03:10:08
4,2021-01-26 04:00:00,11240,Graz/Flughafen,340,2021-01-26,04:00,-4.7,-5.3,96,280,1.8,null,9.4,0.0,1012.0,966.6,0,2021-01-26 04:10:09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6693,2021-11-03 18:00:00,11240,Graz/Flughafen,340,2021-11-03,18:00,7.9,7.3,96,160,5.4,null,9.4,0.2,1005.3,962.2,0,2021-11-03 18:10:10
6694,2021-11-03 19:00:00,11240,Graz/Flughafen,340,2021-11-03,19:00,7.6,7.0,96,230,5.4,null,9.4,0.5,1004.7,961.6,0,2021-11-03 19:10:08
6695,2021-11-03 20:00:00,11240,Graz/Flughafen,340,2021-11-03,20:00,7.6,7.0,96,0,1.8,null,9.4,0.9,1004.2,961.1,0,2021-11-03 20:10:12
6696,2021-11-03 21:00:00,11240,Graz/Flughafen,340,2021-11-03,21:00,7.9,7.5,97,80,3.6,null,7.6,1.0,1003.8,960.8,0,2021-11-03 21:10:08


* Deleting the timestamps column, zatum and zeit 

In [29]:
WeatDataFrame.drop('datum', inplace = True, axis=1) 
WeatDataFrame.drop('zeit', inplace = True, axis=1) 
WeatDataFrame.drop('timestamp', inplace = True, axis=1) 

Save the data frame into a csv file 

In [30]:
WeatDataFrame.to_csv("./DataSets/Raw_Weather_DataSet.csv",index=False)

### Forecast 
#### API 

We'll use python to request de dataset from the OpenWeather data base. The API  request call should be in this format: 

``` http://history.openweathermap.org/data/2.5/history/city?q={city ID},{country code}&type=hour&start={start}&end={end}&appid={API key}```

#### Usage 

Once we introduce the day that we want to predict we request for that day's forecast. If we have it, we arrange all the necessarry inputs for that day (temp, holiday, week day,etc) and we introduce them into the model. 

In [3]:
URL = " http://api.openweathermap.org/data/2.5/forecast?q=Graz&appid=" + API_KEY
response = rq.request("GET", URL)
print(response.json()["list"])

[{'dt': 1637852400, 'main': {'temp': 276.43, 'feels_like': 276.43, 'temp_min': 276.43, 'temp_max': 276.47, 'pressure': 1010, 'sea_level': 1010, 'grnd_level': 966, 'humidity': 57, 'temp_kf': -0.04}, 'weather': [{'id': 802, 'main': 'Clouds', 'description': 'scattered clouds', 'icon': '03d'}], 'clouds': {'all': 31}, 'wind': {'speed': 0.4, 'deg': 341, 'gust': 0.55}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'd'}, 'dt_txt': '2021-11-25 15:00:00'}, {'dt': 1637863200, 'main': {'temp': 275.99, 'feels_like': 275.99, 'temp_min': 275.78, 'temp_max': 275.99, 'pressure': 1009, 'sea_level': 1009, 'grnd_level': 965, 'humidity': 63, 'temp_kf': 0.21}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04n'}], 'clouds': {'all': 53}, 'wind': {'speed': 1.09, 'deg': 301, 'gust': 1.05}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'n'}, 'dt_txt': '2021-11-25 18:00:00'}, {'dt': 1637874000, 'main': {'temp': 276.02, 'feels_like': 276.02, 'temp_min': 276.02, 'temp_max': 276.

In [4]:
response.json()["list"]

[{'dt': 1637852400,
  'main': {'temp': 276.43,
   'feels_like': 276.43,
   'temp_min': 276.43,
   'temp_max': 276.47,
   'pressure': 1010,
   'sea_level': 1010,
   'grnd_level': 966,
   'humidity': 57,
   'temp_kf': -0.04},
  'weather': [{'id': 802,
    'main': 'Clouds',
    'description': 'scattered clouds',
    'icon': '03d'}],
  'clouds': {'all': 31},
  'wind': {'speed': 0.4, 'deg': 341, 'gust': 0.55},
  'visibility': 10000,
  'pop': 0,
  'sys': {'pod': 'd'},
  'dt_txt': '2021-11-25 15:00:00'},
 {'dt': 1637863200,
  'main': {'temp': 275.99,
   'feels_like': 275.99,
   'temp_min': 275.78,
   'temp_max': 275.99,
   'pressure': 1009,
   'sea_level': 1009,
   'grnd_level': 965,
   'humidity': 63,
   'temp_kf': 0.21},
  'weather': [{'id': 803,
    'main': 'Clouds',
    'description': 'broken clouds',
    'icon': '04n'}],
  'clouds': {'all': 53},
  'wind': {'speed': 1.09, 'deg': 301, 'gust': 1.05},
  'visibility': 10000,
  'pop': 0,
  'sys': {'pod': 'n'},
  'dt_txt': '2021-11-25 18:00:00'

In [70]:
df= pd.DataFrame(response.json()['list'])

In [71]:
df.head()

,dt,main,weather,clouds,wind,visibility,pop,sys,dt_txt,rain
0,1637582400,"{'temp': 278.73, 'feels_like': 278.73, 'temp_m...","[{'id': 803, 'main': 'Clouds', 'description': ...",{'all': 83},"{'speed': 0.7, 'deg': 308, 'gust': 1.61}",10000,0.16,{'pod': 'd'},2021-11-22 12:00:00,NaN
1,1637593200,"{'temp': 279.18, 'feels_like': 279.18, 'temp_m...","[{'id': 804, 'main': 'Clouds', 'description': ...",{'all': 92},"{'speed': 0.48, 'deg': 2, 'gust': 1.34}",10000,0.09,{'pod': 'd'},2021-11-22 15:00:00,NaN
2,1637604000,"{'temp': 278.91, 'feels_like': 278.91, 'temp_m...","[{'id': 804, 'main': 'Clouds', 'description': ...",{'all': 100},"{'speed': 0.31, 'deg': 127, 'gust': 1.55}",10000,0.25,{'pod': 'n'},2021-11-22 18:00:00,NaN
3,1637614800,"{'temp': 278.48, 'feels_like': 278.48, 'temp_m...","[{'id': 804, 'main': 'Clouds', 'description': ...",{'all': 100},"{'speed': 0.58, 'deg': 326, 'gust': 0.99}",10000,0.22,{'pod': 'n'},2021-11-22 21:00:00,NaN
4,1637625600,"{'temp': 277.29, 'feels_like': 277.29, 'temp_m...","[{'id': 804, 'main': 'Clouds', 'description': ...",{'all': 99},"{'speed': 1.16, 'deg': 343, 'gust': 1.39}",10000,0.10,{'pod': 'n'},2021-11-23 00:00:00,NaN


* Creating a new data frame with only: 

 * Temperature date 
 * Timestamps (not Unix Time but string dates) 

In [72]:
temp = [] 
timestamp=  [] 
FDf = pd.DataFrame()
for idx,value in enumerate(df['main']): 
    #print(value['temp'])
    #print(df['dt_txt'][idx])
    temp.append(value['temp']-273.15) #Kelvin to celsius 
    timestamp.append(df['dt_txt'][idx])

In [73]:
FDf.insert(0,'Time',timestamp)

In [74]:
FDf.insert(1,'t',temp)

In [75]:
FDf.head()

,Time,t
0,2021-11-22 12:00:00,5.58
1,2021-11-22 15:00:00,6.03
2,2021-11-22 18:00:00,5.76
3,2021-11-22 21:00:00,5.33
4,2021-11-23 00:00:00,4.14


 * Converting into datetime type objects

In [76]:
from datetime import datetime as dt
def Convertstr2dt(data,col): 
    dates=[]
    for str_date in data[col]: 
        dti = dt.strptime(str_date, "%Y-%m-%d %H:%M:%S")
        dates.append(dti)
   
    #Delete the old column
    data.drop(col, inplace = True, axis=1) 

    #Insert the newone
    data.insert(0, col,dates)
    return data

DataFrame = Convertstr2dt(FDf,'Time')

In [77]:
DataFrame.head()

,Time,t
0,2021-11-22 12:00:00,5.58
1,2021-11-22 15:00:00,6.03
2,2021-11-22 18:00:00,5.76
3,2021-11-22 21:00:00,5.33
4,2021-11-23 00:00:00,4.14


### Adding the others inputs columns

We create the whole input dataframe matching these forecast days. 

* Identify the week days

In [78]:
weekday = [] 
for day in DataFrame['Time']: 
    weekday.append(day.weekday())

#Add the new column
DataFrame.insert(1, 'Week Day',weekday)
DataFrame.head()

,Time,Week Day,t
0,2021-11-22 12:00:00,0,5.58
1,2021-11-22 15:00:00,0,6.03
2,2021-11-22 18:00:00,0,5.76
3,2021-11-22 21:00:00,0,5.33
4,2021-11-23 00:00:00,1,4.14


#### Decoding the weekdays

In [79]:
import math
WeekDaysNorm = 2 * math.pi * DataFrame["Week Day"] / (DataFrame["Week Day"].max()+1) #+1 cause we start from 0

DataFrame["cos_x"] = np.cos(WeekDaysNorm)
DataFrame["sin_x"] = np.sin(WeekDaysNorm)

DataFrame.head(100)

,Time,Week Day,t,cos_x,sin_x
0,2021-11-22 12:00:00,0,5.58,1.0,0.000000e+00
1,2021-11-22 15:00:00,0,6.03,1.0,0.000000e+00
2,2021-11-22 18:00:00,0,5.76,1.0,0.000000e+00
3,2021-11-22 21:00:00,0,5.33,1.0,0.000000e+00
4,2021-11-23 00:00:00,1,4.14,0.5,8.660254e-01
5,2021-11-23 03:00:00,1,3.60,0.5,8.660254e-01
6,2021-11-23 06:00:00,1,2.94,0.5,8.660254e-01
7,2021-11-23 09:00:00,1,5.82,0.5,8.660254e-01
8,2021-11-23 12:00:00,1,7.67,0.5,8.660254e-01
9,2021-11-23 15:00:00,1,4.97,0.5,8.660254e-01


#### Holidays matching

In [80]:
from datetime import date 
import holidays 
holidaysList = [] 

for day, name in sorted(holidays.AT(state='9', years=2021).items()): 
    holidaysList.append(day)
    
print(holidaysList, len(holidaysList))

DataFrame.insert(1,"Holiday",np.zeros(len(DataFrame)))
for i,date in enumerate(DataFrame['Time']):
    if date in holidaysList: 
        DataFrame["Holiday"].loc[i] = 1 
        


[datetime.date(2021, 1, 1), datetime.date(2021, 1, 6), datetime.date(2021, 4, 5), datetime.date(2021, 5, 1), datetime.date(2021, 5, 13), datetime.date(2021, 5, 24), datetime.date(2021, 6, 3), datetime.date(2021, 8, 15), datetime.date(2021, 10, 26), datetime.date(2021, 11, 1), datetime.date(2021, 12, 8), datetime.date(2021, 12, 25), datetime.date(2021, 12, 26)] 13


In [81]:
def SaveDataFrame(data, name): 
    data.to_csv(name,index=False)
    return 

In [83]:
SaveDataFrame(DataFrame,"./DataSets/Forecast_DataSet.csv")